In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Projects/TextMining/Text_Classifier1/DeepLearningTrain')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pickle
import sqlite3
import numpy as np
import pandas as pd

In [3]:
def split_spaces(data): ## 输入未处理的数据，第二列为文本，返回分割好第二列的列表。
    data_list = []
    for line in data:
        data_list.append(list(line))
        
    for item in data_list:
        item[1] = ' '.join(list(item[1]))
    return data_list

def desymbol(data):
    import re
    data_desymbol = []
    for line in data:
        data_desymbol.append(list(line))
    for row in data_desymbol:
        row[1] = ' '.join(list(re.sub("[\s+\.\!\/_,$%^*)(+\"\']+|[+——！，。？：、~@#￥%……&*（）“”]", "",row[1])))
    return data_desymbol

def add_start_end(data):
    data_with_se = []
    for line in data:
        data_with_se.append(list(line))
    for row in data_with_se:
        row[1] = "<start> "+row[1]+" <end>"
    return data_with_se

def countV(x_test):
    xtest_cv = cv.transform(x_test)
    return xtest_cv

def tfIdfV(x_test):
    x_test_tfidf = tv.transform(x_test)
    
    return x_test_tfidf

In [4]:
def load_from_db(num,start):
    conn = sqlite3.connect("../data/database/texts.db")
    print ('Opened database successfully')
    c = conn.cursor()

    ci_data=[]
    cursor = c.execute("SELECT * from ci order by id desc limit "+str(start)+","+str(num))
    for row in cursor:
        ci_data.append(row)

    poet_data = []
    cursor=c.execute("SELECT * FROM poet order by id desc limit "+str(start)+","+str(num))
    for row in cursor:
        poet_data.append(row)

    classical_data = []
    cursor=c.execute("SELECT * FROM classical order by id desc limit "+str(start)+","+str(num))
    for row in cursor:
        classical_data.append(row)

    journal_data = []
    cursor=c.execute("SELECT * FROM journal order by id desc limit "+str(start)+","+str(num))
    for row in cursor:
        journal_data.append(row)
    
    news_data = []
    cursor=c.execute("SELECT * FROM news order by id asc limit "+str(start)+","+str(num))
    for row in cursor:
        news_data.append(row)

    print("所有类型数据读取成功，数量"+str(num))
    data = ci_data + poet_data + classical_data + journal_data + news_data
    
    return add_start_end(desymbol(split_spaces(data)))


In [5]:
file = open("../model/CV.pkl", "rb")
cv = pickle.load(file)
file.close()

file = open("../model/TV.pkl", "rb")
tv = pickle.load(file)
file.close()

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
def get_text(data):
    textlist = []
    if(len(np.array(data).shape) > 1):
        for row in data:
            textlist.append(row[1])
    else: textlist.append(data[1])
    return textlist

In [8]:
data = load_from_db(2000,0)
textlist = get_text(data)
# c = {"text":textlist,"cv":list(countV(textlist)),"tfIdf":list(tfIdfV(textlist))}
# df = pd.DataFrame(c)

# display(df)
# print(tfIdfV(textlist).shape)
# print(list(tfIdfV(textlist))[5])
test_data = load_from_db(200,18000)
test_textlist = get_text(data)


Opened database successfully
所有类型数据读取成功，数量2000
Opened database successfully
所有类型数据读取成功，数量200


In [9]:
y_train = []
for row in data:
    y_train.append(row[2])
print(y_train[0])
for i in range(len(y_train)):
    if y_train[i] =="modern":
        y_train[i]=0
    elif y_train[i] =="ci":
        y_train[i]=1
    elif y_train[i] =="poet":
        y_train[i]=1
    elif y_train[i] =="classical":
        y_train[i]=1
print(y_train[0])

ci
1


In [10]:
y_test = []
for row in test_data:
    y_test.append(row[2])
print(y_test[0])
for i in range(len(y_test)):
    if y_test[i] =="modern":
        y_test[i]=0
    elif y_test[i] =="ci":
        y_test[i]=1
    elif y_test[i] =="poet":
        y_test[i]=1
    elif y_test[i] =="classical":
        y_test[i]=1
print(y_test[0])

ci
1


In [11]:
x_train_cv = countV(textlist)
x_train_tfidf = tfIdfV(textlist)

In [12]:
x_test_cv = countV(textlist)

In [15]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import sklearn
import string
import random
# nltk.download('names')
# from nltk.corpus import names

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

'''
将名字编码为向量：每个字符为one-hot编码，将多个字符的向量进行堆叠
abc = [ [1, 0, ...,0]
        [0, 1, 0, ..]
        [0, 0, 1, ..] ]
abc.shape = (len("abc"), len(chars))
'''
# def name2vec(name):
#     ids = [chars.index(c) for c in name if c not in ["\\"]]

#     a = np.zeros(shape=(len(ids), len(chars)))
#     for i, idx in enumerate(ids):
#         a[i][idx] = 1
#     return a


# def load_data():
#     female_file, male_file = names.fileids()

#     f1_names = names.words(female_file)
#     f2_names = names.words(male_file)

#     data_set = [(name.lower(), 0) for name in f1_names] + [(name.lower(), 1) for name in f2_names]
#     data_set = [(name2vec(name), sexy) for name, sexy in data_set]
#     random.shuffle(data_set)
#     return data_set


class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.rnn = nn.RNN(vocab_size, hidden_size, batch_first=True)
        self.liner = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        h0 = torch.zeros(1, 1, self.hidden_size, device=device) # 初始hidden state
        output, hidden = self.rnn(input, h0)
        output = output[:, -1, :] # 只使用最终时刻的输出作为特征
        output = self.liner(output)
        output = F.softmax(output, dim=1)
        return output

hidden_dim = 128
output_dim = 2

class Model:
    def __init__(self, epoches=100):
        self.model = CharRNN(20000, hidden_dim , output_dim)
        self.model.to(device)
        self.epoches = epoches

    def train(self,x_train,y_train):
        loss_func = nn.CrossEntropyLoss()
        optimizer = torch.optim.RMSprop(self.model.parameters(), lr=0.0003)

        for epoch in range(self.epoches):
            total_loss = 0
            for x in range(len(y_train)):# 每轮随机样本训练1000次
#                 name, sexy = random.choice(train_set)
                texts , typetext = x_train[x],y_train[x]
#                 # RNN的input要求shape为[batch, seq_len, embed_dim]，由于名字为变长，也不准备好将其填充为定长，因此batch_size取1，将取的名字放入单个元素的list中。
#                 name_tensor = torch.tensor([name], dtype=torch.float, device=device)
                texts_tensor = torch.tensor([texts.toarray()], dtype=torch.float,device=device)
                typetext_tensor = torch.tensor([typetext], dtype=torch.long, device=device)
#                 # torch要求计算损失时，只提供类别的索引值，不需要one-hot表示
#                 sexy_tensor = torch.tensor([sexy], dtype=torch.long, device=device)

                optimizer.zero_grad()

                pred = self.model(texts_tensor) # [batch, out_dim]
                loss = loss_func(pred, typetext_tensor)
                loss.backward()
                total_loss += loss
                optimizer.step()
            print("Training: in epoch {} loss {}".format(epoch, total_loss/1000))

    def evaluate(self,x_test,y_test):
        with torch.no_grad(): # 评估时不进行梯度计算
            correct = 0
            for x in range(len(y_test)): # 从测试集中随机采样测试1000次
#                 name, sexy = random.choice(test_set)
#                 name_tensor = torch.tensor([name], dtype=torch.float, device=device)
                texts , typetext = x_test[x],y_test[x] 
                texts_tensor = torch.tensor([texts.toarray()], dtype=torch.float,device=device)
                
                pred = self.model(texts_tensor)
                if torch.argmax(pred).item() == typetext:
                    correct += 1

            print('Evaluating: test accuracy is {}%'.format(correct*100/len(y_test)))

#     def predict(self, name):
#         p = name2vec(name.lower())
#         name_tensor = torch.tensor([p], dtype=torch.float, device=device)
#         with torch.no_grad():
#             out = self.model(name_tensor)
#             out = torch.argmax(out).item()
#             sexy = 'female' if out == 0 else 'male'
#             print('{} is {}'.format(name, sexy))


model = Model(10)
#     data_set = load_data()
#     train, test = sklearn.model_selection.train_test_split(data_set)
model.train(x_train_cv,y_train)
model.evaluate(x_test_cv,y_test)

Training: in epoch 0 loss 4.837851524353027
Training: in epoch 1 loss 4.4026899337768555
Training: in epoch 2 loss 3.203468084335327
Training: in epoch 3 loss 3.1627461910247803
Training: in epoch 4 loss 3.1523489952087402
Training: in epoch 5 loss 3.1426098346710205
Training: in epoch 6 loss 3.1383049488067627
Training: in epoch 7 loss 3.1390655040740967
Training: in epoch 8 loss 3.1388285160064697
Training: in epoch 9 loss 3.1343703269958496
Evaluating: test accuracy is 6000.0%


In [ ]:
print(x_train_cv[0])
print(x_train_cv.toarray()[0])

  (0, 2)	8
  (0, 195)	1
  (0, 256)	1
  (0, 319)	1
  (0, 629)	1
  (0, 632)	1
  (0, 638)	1
  (0, 640)	1
  (0, 644)	1
  (0, 667)	1
  (0, 669)	1
  (0, 676)	1
  (0, 680)	1
  (0, 684)	1
  (0, 685)	2
  (0, 686)	1
  (0, 2647)	1
  (0, 4156)	1
  (0, 5340)	1
  (0, 8267)	1
  (0, 10130)	1
  (0, 12559)	1
  (0, 12781)	1
[0 0 8 ... 0 0 0]
